In [5]:
import io
from IPython.nbformat import current

def import_notebook(nbfile, dunder_name=None):
    import sys
    import imp
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')

    if dunder_name is None:
        dunder_name = ".".join(nbfile.split(".")[:-1])
        
    if dunder_name in sys.modules:
        print "Module already imported. Returning existing module."
        return sys.modules[dunder_name]

    print "Importing ipynb file : {}".format(nbfile)
        
    newModule = imp.new_module(dunder_name)
    newModule.__file__ = "ipython_notebooks/{}".format(nbfile)

    try:
        for cell in nb.worksheets[0].cells:
            if cell.cell_type != 'code':
                continue
            exec cell.input in newModule.__dict__
    finally:
        print "Finished importing."
    sys.modules[dunder_name] = newModule
    return newModule

etools = import_notebook("Euler_Tools.ipynb")


Importing ipynb file : Euler_Tools.ipynb
Module already imported. Returning existing module.


### Problem 21

Let d(n) be defined as the sum of proper divisors of n (numbers less than n which divide evenly into n).
If d(a) = b and d(b) = a, where a ≠ b, then a and b are an amicable pair and each of a and b are called amicable numbers.

For example, the proper divisors of 220 are 1, 2, 4, 5, 10, 11, 20, 22, 44, 55 and 110; therefore d(220) = 284. The proper divisors of 284 are 1, 2, 4, 71 and 142; so d(284) = 220.

Evaluate the sum of all the amicable numbers under 10000.

In [2]:
from itertools import combinations

def getExpandedFactors(n, primes=None):
    factors = []
    [factors.extend([x[0]]*x[1]) for x in factorize(n, primes)]
    return factors

def genDivisors(num, primes=None):
    if num > 0:
        yield 1
    factors = getExpandedFactors(num, primes)
    for ix in xrange(1, len(factors)):
        for terms in combinations(factors, ix):
            yield product(terms)

sumDivs = {}

for num in xrange(1, 10001):
    sumDivs[num] = sum(set(genDivisors(num)))

amicables = []
    
for key, val in sumDivs.items():
    if (key == val) or (val < key):
        continue
    if sumDivs.get(val) == key:
        amicables.extend([key, val])

print amicables
print sum(amicables)

NameError: global name 'factorize' is not defined

### Problem 22

Using names.txt (right click and 'Save Link/Target As...'), a 46K text file containing over five-thousand first names, begin by sorting it into alphabetical order. Then working out the alphabetical value for each name, multiply this value by its alphabetical position in the list to obtain a name score.

For example, when the list is sorted into alphabetical order, COLIN, which is worth 3 + 15 + 12 + 9 + 14 = 53, is the 938th name in the list. So, COLIN would obtain a score of 938 × 53 = 49714.

What is the total of all the name scores in the file?

In [ ]:
# import os
# os.system("wget https://projecteuler.net/project/resources/p022_names.txt\
# --no-check-certificate")

def getScore(name, index):
    ordbase = ord('a') - 1
    return index * sum((ord(c) - ordbase) for c in name)

names = (x.strip().lower() for x in open('p022_names.txt').read().replace('"',
                                      '').split(','))
print sum(getScore(cc, ix + 1) for ix, cc in enumerate(sorted(names)))


### Problem 23

A perfect number is a number for which the sum of its proper divisors is exactly equal to the number. For example, the sum of the proper divisors of 28 would be 1 + 2 + 4 + 7 + 14 = 28, which means that 28 is a perfect number.

A number n is called deficient if the sum of its proper divisors is less than n and it is called abundant if this sum exceeds n.

As 12 is the smallest abundant number, 1 + 2 + 3 + 4 + 6 = 16, the smallest number that can be written as the sum of two abundant numbers is 24. By mathematical analysis, it can be shown that all integers greater than 28123 can be written as the sum of two abundant numbers. However, this upper limit cannot be reduced any further by analysis even though it is known that the greatest number that cannot be expressed as the sum of two abundant numbers is less than this limit.

Find the sum of all the positive integers which cannot be written as the sum of two abundant numbers.

In [1]:
def sumOfDivisors(num, primes=None):
    return sum(set(genDivisors(num, primes)))

def isAbundant(num, primes=None):
    return True if sumOfDivisors(num, primes) > num else False

def isPerfect(num, primes=None):
    return True if sumOfDivisors(num, primes) == num else False

@timeit
def solve():
    primes = list(genPrimes(29000))
    abundants = [x for x in xrange(2, 28124) if isAbundant(x, primes)]
    labu = len(abundants)
    pairSums = set((abundants[i] + abundants[j]) for i in xrange(labu)
                           for j in xrange(i, labu))
    
    pairSums = sum(x for x in pairSums if x < 28124)

    print (28123 * 28124 / 2) - pairSums
    

solve()

Time = 9.05990600586e-06


NameError: global name 'genPrimes' is not defined

### Problem 24

A permutation is an ordered arrangement of objects. For example, 3124 is one possible permutation of the digits 1, 2, 3 and 4. If all of the permutations are listed numerically or alphabetically, we call it lexicographic order. The lexicographic permutations of 0, 1 and 2 are:

012   021   102   120   201   210

What is the millionth lexicographic permutation of the digits 0, 1, 2, 3, 4, 5, 6, 7, 8 and 9?

In [ ]:
def findSmallestHigherIndex(inp, index):
    prev = inp[index - 1]
    remaining = inp[index:]
    minx = remaining[0]
    minix = index
    for ix, x in enumerate(remaining):
        if (x > prev) and (x < minx):
            minx = x
            minix = ix + index
    return minix
    

def genPermutations(inp):
    inp = sorted(inp)
    yield inp
    # read numbers from right to left
    # if increasing, continue
    #     if reach beginning, return
    # if decreasing, exchange lesser number with smallest higher number on right
    # sort all numbers on the right,
    # yield permutation
    # back to first step
    linp = len(inp)
    while True:
        for index in xrange(linp - 1, -1, -1):
            if index == 0:
                return
            if inp[index - 1] > inp[index]:
                continue
            else:
                idx = findSmallestHigherIndex(inp, index)
                inp[index - 1], inp[idx] = inp[idx], inp[index - 1]
                inp[index:] = sorted(inp[index:])
                yield inp
                break
    return

@timeit
def solve():
    for ix, x in enumerate(genPermutations(range(10))):
        if ix == 999999:
            print "".join(str(y) for y in x)
            break
            
solve()

### Problem 25

The Fibonacci sequence is defined by the recurrence relation:

Fn = Fn−1 + Fn−2, where F1 = 1 and F2 = 1.
Hence the first 12 terms will be:

```
F1 = 1
F2 = 1
F3 = 2
F4 = 3
F5 = 5
F6 = 8
F7 = 13
F8 = 21
F9 = 34
F10 = 55
F11 = 89
F12 = 144
```
The 12th term, F12, is the first term to contain three digits.

What is the index of the first term in the Fibonacci sequence to contain 1000 digits?

In [ ]:
def genFibonacci():
    a, b = 1, 1
    yield 1
    yield 1
    while True:
        a, b = b, a + b
        yield b

@timeit
def solve():
    for ix, x in enumerate(genFibonacci()):
        if len(str(x)) >= 1000:
            print ix + 1
            break

solve()


### Problem 26

A unit fraction contains 1 in the numerator. The decimal representation of the unit fractions with denominators 2 to 10 are given:
```
1/2	= 	0.5
1/3	= 	0.(3)
1/4	= 	0.25
1/5	= 	0.2
1/6	= 	0.1(6)
1/7	= 	0.(142857)
1/8	= 	0.125
1/9	= 	0.(1)
1/10	= 	0.1
```
Where 0.1(6) means 0.166666..., and has a 1-digit recurring cycle. It can be seen that 1/7 has a 6-digit recurring cycle.

Find the value of d < 1000 for which 1/d contains the longest recurring cycle in its decimal fraction part.

In [ ]:
def seqLength(num):
    rems = [1]
    while True:
        nxt = (rems[-1] * 10) % num
        if nxt in rems:
            return len(rems) - rems.index(nxt) 
        else:
            rems.append(nxt)
    return

result = sorted(((seqLength(x), x) for x in xrange(2, 1000)), reverse=True)[:10]

print result
print result[0][1]

### Problem 27

Euler discovered the remarkable quadratic formula:

n² + n + 41

It turns out that the formula will produce 40 primes for the consecutive values n = 0 to 39. However, when n = 40, 402 + 40 + 41 = 40(40 + 1) + 41 is divisible by 41, and certainly when n = 41, 41² + 41 + 41 is clearly divisible by 41.

The incredible formula  n² − 79n + 1601 was discovered, which produces 80 primes for the consecutive values n = 0 to 79. The product of the coefficients, −79 and 1601, is −126479.

Considering quadratics of the form:

n² + an + b, where |a| < 1000 and |b| < 1000

where |n| is the modulus/absolute value of n
e.g. |11| = 11 and |−4| = 4
Find the product of the coefficients, a and b, for the quadratic expression that produces the maximum number of primes for consecutive values of n, starting with n = 0.

In [ ]:
bVals = getPrimes(1000)
primes = set(genPrimesNP(1000000))


In [ ]:
def nval(n, a, b):
    return (n*n + a*n + b)

maxn = 0
maxnab = None

arange = 1000

for b in bVals:
    for a in xrange(-arange, arange + 1):
        for n in xrange(0, 10000):
            if nval(n, a, b) not in primes:
                n = n - 1
                if n > maxn:
                    maxn = n
                    maxnab = (n, a, b)
                break

print maxnab
print maxnab[1]*maxnab[2]


### Problem 28

Starting with the number 1 and moving to the right in a clockwise direction a 5 by 5 spiral is formed as follows:

```
21 22 23 24 25
20  7  8  9 10
19  6  1  2 11
18  5  4  3 12
17 16 15 14 13
```

It can be verified that the sum of the numbers on the diagonals is 101.

What is the sum of the numbers on the diagonals in a 1001 by 1001 spiral formed in the same way?

In [ ]:
@timeit
def genDiagonals():
    num = 1
    yield num
    for x in [[y]*4 for y in range(2, 1001, 2)]:
        for z in x:
            num += z
            yield num

print sum(genDiagonals())

### Problem 29

Consider all integer combinations of $a^b$ for 2 ≤ a ≤ 5 and 2 ≤ b ≤ 5:

$$2^2=4, 2^3=8, 2^4=16, 2^5=32$$
$$3^2=9, 3^3=27, 3^4=81, 3^5=243$$
$$4^2=16, 4^3=64, 4^4=256, 4^5=1024$$
$$5^2=25, 5^3=125, 5^4=625, 5^5=3125$$

If they are then placed in numerical order, with any repeats removed, we get the following sequence of 15 distinct terms:

4, 8, 9, 16, 25, 27, 32, 64, 81, 125, 243, 256, 625, 1024, 3125

How many distinct terms are in the sequence generated by ab for 2 ≤ a ≤ 100 and 2 ≤ b ≤ 100?


In [ ]:
print len(set(pow(a,b) for a in xrange(2, 101) for b in xrange(2, 101)))
        

### Problem 30

Surprisingly there are only three numbers that can be written as the sum of fourth powers of their digits:

```
1634 = 14 + 64 + 34 + 44
8208 = 84 + 24 + 04 + 84
9474 = 94 + 44 + 74 + 44
```

As $1 = 1^4$ is not a sum it is not included.

The sum of these numbers is 1634 + 8208 + 9474 = 19316.

Find the sum of all the numbers that can be written as the sum of fifth powers of their digits.

In [ ]:
def sumPowers(num):
    nums = str(num)
    ord0 = ord('0')
    numa = ((ord(x) - ord0) for x in nums)
    return sum(pow(x, 5) for x in numa)

print sum(x for x in xrange(2, 6*pow(9, 5)) if x == sumPowers(x))



### Problem 31

In England the currency is made up of pound, £, and pence, p, and there are eight coins in general circulation:

1p, 2p, 5p, 10p, 20p, 50p, £1 (100p) and £2 (200p).
It is possible to make £2 in the following way:

1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p
How many different ways can £2 be made using any number of coins?

In [ ]:
values = [1, 2, 5, 10, 20, 50, 100, 200]
vset = set(values)
cache = {}

numCalls = 0

def merge(a, b):
    i, j = 0, 0
    result = []
    while True:
        if i == len(a):
            result.extend(b[j:])
            break
        if j == len(b):
            result.extend(a[i:])
            break;
        if a[i] >= b[j]:
            result.append(a[i])
            i += 1
        else:
            result.append(b[j])
            j += 1
    return tuple(result)
            
def getDecoms(n):
    global numCalls
    
    if n in cache:
        return cache[n]
    
    if n == 0:
        cache[0] = set()
        return set()
    if n == 1:
        cache[n] = set([(n, )])
        return cache[n]

    ways = set()
    for k in values:
        if k > n:
            break
        result1 = getDecoms(n-k)
        if len(result1) == 0:
            ways.add((k, ))
        else:
            for r1 in result1:
                if k >= r1[0]:
                    numCalls += 1
                    ways.add(merge((k, ), r1))
    cache[n] = ways
    return ways

@timeit
def findResult(n):
    return getDecoms(n)

result = findResult(100)

print "numcalls = {}".format(numCalls)
print len(result), "\n", sorted(result, reverse=True)[:20]



In [ ]:
coins = [2000, 1000, 500, 200, 100, 50, 20, 10, 5, 2, 1]
cache = {}

def findPoss(money, maxCoin):
    if (money, maxCoin) in cache:
        return cache[(money, maxCoin)]
    sum = 0
    if maxCoin == len(coins) - 1:
        return 1
    for xx in range(maxCoin, len(coins)):
        diff = money - coins[xx]
        if diff == 0:
            sum += 1
        elif diff > 0:
            sum += findPoss(diff, xx)
    cache[(money, maxCoin)] = sum
    return sum

@timeit
def solve(money, index):
    return findPoss(money, index)

print solve(2000, 0)

##### Problem 32

We shall say that an n-digit number is pandigital if it makes use of all the digits 1 to n exactly once; for example, the 5-digit number, 15234, is 1 through 5 pandigital.

The product 7254 is unusual, as the identity, 39 × 186 = 7254, containing multiplicand, multiplier, and product is 1 through 9 pandigital.

Find the sum of all products whose multiplicand/multiplier/product identity can be written as a 1 through 9 pandigital.

HINT: Some products can be obtained in more than one way so be sure to only include it once in your sum.

In [1]:
from itertools import permutations

products = set()

@timeit
def solve():
    digits = set([str(x) for x in range(1, 10)])
    for xr, yr in [(3, 2), (4, 1)]:
        for xx in permutations(digits, xr):
            if xx[-1] == '5':
                continue
            remaining = digits.difference(set(xx))
            ixx = int("".join(xx))
            for yy in permutations(remaining, yr):
                if yy[-1] == '5':
                    continue
                iyy = int("".join(yy))
                prod = str(ixx*iyy)
                if len(prod) != 4:
                    continue
                sprod = remaining.difference(set(yy))
                if sprod == set(prod):
                    products.add(ixx*iyy)
                
    return products

result = solve()

pp(result)
print sum(result)

                

Time = 0.0404181480408
set([4396, 5346, 5796, 6952, 7254, 7632, 7852])
45228


### Problem 33

The fraction 49/98 is a curious fraction, as an inexperienced mathematician in attempting to simplify it may incorrectly believe that 49/98 = 4/8, which is correct, is obtained by cancelling the 9s.

We shall consider fractions like, 30/50 = 3/5, to be trivial examples.

There are exactly four non-trivial examples of this type of fraction, less than one in value, and containing two digits in the numerator and denominator.

If the product of these four fractions is given in its lowest common terms, find the value of the denominator.

In [9]:
fractions = []

@timeit
def solve():
    for x in range(11, 100):
        if x % 10 == 0:
            continue
        for y in range(x+1, 100):
            if y % 10 == 0:
                continue

            x1, x2 = x/10, x%10
            y1, y2 = y/10, y%10

            if x1 == y2 and x2 == y1:
                continue
            r1 = float(x)/y
            r2 = None
            if x1 == y2:
                r2 = float(x2) / y1
            elif x2 == y1:
                r2 = float(x1) / y2

            if r1 == r2:
                fractions.append((x, y))

    num = product((x[0] for x in fractions))
    denom = product((x[1] for x in fractions))

    for x in range(2, 100):
        while (num % x == 0) and (denom % x == 0):
            num = num / x
            denom = denom / x
        if x > num:
            break
            
    return fractions, num, denom

frac, num, denom = solve()

print frac
print num, denom
            
            

Time = 0.00310778617859
[(16, 64), (19, 95), (26, 65), (49, 98)]
1 100


### Problem 34

145 is a curious number, as 1! + 4! + 5! = 1 + 24 + 120 = 145.

Find the sum of all numbers which are equal to the sum of the factorial of their digits.

Note: as 1! = 1 and 2! = 2 are not sums they are not included.

In [12]:
cache = {}

def doesItSatCrit(n):
    k = n
    ksum = 0
    while k > 0:
        if k in cache:
            ksum += cache[k]
            break
        ksum += factorial(k % 10)
        k = k / 10
    cache[n] = ksum
    if ksum == n:
        return True
    return False

for x in xrange()

3628800


In [24]:
print 5*factorial(9)

1814400
